# Part 5: Integrating Multiple Modalities

**Tutor:** Anthony Christidis
**Time:** 30 minutes

---

In our final notebook, we will tackle one of the most powerful and scientifically relevant challenges in spatial biology: integrating data from multiple technologies. We have two datasets from the same block of human breast cancer tissue:

1. A **Visium** dataset, providing a whole-transcriptome view at spot-level resolution.
2. A **Xenium** dataset, providing a high-resolution view of ~300 targeted genes at the single-cell level.

Our goal is to align these two datasets into a single, shared coordinate system. This will allow us to, for example, analyze the full transcriptome of Visium spots that fall within a specific high-resolution cell type niche defined by Xenium.

**Goals:**
1. Understand the concept of coordinate systems in `SpatialData`.
2. Use `napari` to interactively define corresponding landmarks on two different images.
3. Apply a transformation to align the datasets into a common coordinate space.
4. Use `sd.aggregate()` to summarize high-resolution data within low-resolution regions.

### Loading the Datasets

First, let's load our two datasets. These are subsets of the data used in the original `nbXX` notebook, cropped to make them manageable for the workshop.

In [ ]:
%load_ext jupyter_black

import spatialdata as sd
import napari_spatialdata as nsd
from spatialdata.transformations import get_transformation_between_landmarks, set_transformation
import warnings

warnings.filterwarnings("ignore")

data_path = "../data/"

sdata_visium = sd.read_zarr(data_path + "visium_breast_cancer_subset.zarr")
sdata_xenium = sd.read_zarr(data_path + "xenium_breast_cancer_subset.zarr")

In [ ]:
print("--- Visium Data ---")
print(sdata_visium)
print("\n--- Xenium Data ---")
print(sdata_xenium)

### Your Turn: Interactive Landmark-Based Alignment

The two datasets are currently in their own separate coordinate systems (`global`). To align them, we need to find common features in both images and mark them as landmarks. `napari` is the perfect tool for this.

**Instructions:**
1. Run the cell below to launch `napari` with both `SpatialData` objects.
2. In the `napari` viewer, you will see two images. Find 3-4 features (like tissue boundaries or specific ducts) that are clearly visible in both the Visium and Xenium images.
3. Add a new `Points` layer for each dataset by clicking the 'Add points layer' button.
4. Carefully click on the same 3-4 landmark locations in each image, adding a point in the corresponding points layer.
5. **Important:** Make sure you add the points in the same order for both images!
6. Leave `napari` open and proceed to the next cell to extract the coordinates.

In [ ]:
# This launches napari with both datasets loaded
viewer = nsd.Interactive([sdata_visium, sdata_xenium])

# After placing your landmarks, run the cells below

Here is an example of what the landmark selection process looks like:

![Landmark selection GIF](https://www.dropbox.com/s/z9d3d3b7v5d7s9t/CleanShot%202023-11-13%20at%2016.37.43.gif?raw=1)

### Calculating and Applying the Transformation

Once we have our landmarks, we can extract their coordinates from the `viewer` object and calculate the transformation matrix needed to align the Visium data to the Xenium data's coordinate space.

In [ ]:
# Note: Layer names might differ slightly. Adjust if necessary.
# Let's assume you created 'Points' and 'Points [1]' layers.
visium_landmark_coords = viewer.window.viewer.layers['Points'].data
xenium_landmark_coords = viewer.window.viewer.layers['Points [1]'].data

# Calculate the affine transformation
affine_transform = get_transformation_between_landmarks(
    moving_coords=visium_landmark_coords,
    references_coords=xenium_landmark_coords
)

# Apply this transformation to all elements in the Visium object,
# creating a new 'aligned_to_xenium' coordinate system.
set_transformation(
    sdata_visium.images['CytAssist_FFPE_Human_Breast_Cancer_full_image'],
    affine_transform, 
    "aligned_to_xenium"
)
set_transformation(
    sdata_visium.shapes['CytAssist_FFPE_Human_Breast_Cancer'],
    affine_transform, 
    "aligned_to_xenium"
)

print("Transformation applied!")
viewer.window.close()

### Visualizing the Aligned Result

Let's use `spatialdata-plot` to visualize our success. We will render the Xenium image and overlay the newly transformed Visium spots.

In [ ]:
sdata_xenium.pl.render_images().pl.show(figsize=(8,8), title="Xenium Image")

# Now render the Visium spots in their new, aligned coordinate system
sdata_visium.pl.render_shapes(
    coordinate_systems="aligned_to_xenium", 
    outline=True, 
    fill_alpha=0.1
).pl.show(figsize=(8,8), title="Visium Spots Aligned to Xenium")

### Final Step: Aggregating Data Across Modalities

Now that the data is aligned, we can perform powerful cross-modality analyses. As a final example, let's use `sd.aggregate()` to count how many high-resolution Xenium cells fall within each low-resolution Visium spot.

In [ ]:
cell_counts_per_spot = sd.aggregate(
    values=sdata_xenium.shapes['cell_boundaries'],
    by=sdata_visium.shapes['CytAssist_FFPE_Human_Breast_Cancer'],
    agg_func="count",
)

# This adds a new column to our Visium table with the cell counts
sdata_visium.tables["table"].obs["xenium_cell_count"] = cell_counts_per_spot["table"].obs["agg_values"]

sdata_visium.tables["table"].obs.head()

Let's visualize the result!

In [ ]:
sdata_visium.pl.render_shapes(
    color="xenium_cell_count",
    size=250,
).pl.show(coordinate_systems="aligned_to_xenium", title="Number of Xenium Cells per Visium Spot")

### Workshop Conclusion

Congratulations! Over the last three hours, you have learned a complete workflow:

1.  How to load and explore complex spatial data with `SpatialData` and `napari`.
2.  How to identify cell types with `scanpy` and analyze their organization with `squidpy`.
3.  How to integrate multiple datasets from different technologies.

You are now equipped with the foundational skills to start applying these powerful `scverse` tools to your own research questions. Thank you for participating!